# Weighted statistical testing

**THIS IS ALL WRONG SINCE IID IS NOT FULFILLED**

Taking the id and comparing the number of values in decile 1-8, shows that there is a huge variance in usable data points between individuals (82-248).

Hence, we need to establish identical weighting.

## Image for prez

In [ ]:
title = "SDRR distribution in PDR patients"
data.sort_values("DPN", inplace=True)
plotter = OverallDetailComparisonPlot(
    data=data[data.id.astype(int) >= 20000000], 
    x_column="id",
    y_column="SDRR",
    hue_column="DPN",
    detail_mode="box",
    points='all',
    title=title)

plotter.fig.update_layout(font=dict(family="Open Sauce", color="black"))


plotter.setup_general_layout(500, 500, 1 / 4)

plotter.show()

plotter.fig.write_image(os.path.join(htmL_save_folder, f"{title}.png"), scale = 6)

In [ ]:
display(data)

,t_start,t_end,SDRR,RMSSD,pNN50 (%),Mean HR (bpm),ULF Power,ULF Peak Frequency,VLF Power,VLF Peak Frequency,...,LF÷HF Ratio_decile,S_decile,SD1_decile,SD2_decile,SD1÷SD2_decile,ApEn_decile,SampEn_decile,DFA α1_decile,DFA α2_decile,D2_decile
0,0.76,600.40,59.95,50.34,4.16,81.75,0.00417,0,3.31,0.03,...,9,1,2,2,3,7,7,8,4,1
3019,16811,17411,74.64,84.44,5.09,92.42,0.00268,0.00167,4.61,0.035,...,1,2,2,1,6,0,1,7,7,1
3018,16211,16811,48.22,43.50,1.92,88.58,0.0015,0.00167,1.06,0.0367,...,9,0,0,0,0,2,2,9,8,0
3017,15611,16211,97.86,102.59,4.40,90.94,0.019,0.00167,10.96,0.0317,...,9,3,3,3,3,0,0,6,5,4
3016,15010,15610,114.53,127.53,12.33,86.03,0.00792,0,14.28,0.0333,...,5,4,4,4,5,1,1,3,6,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4201,2403,3002,22.84,14.53,0.10,99.24,0.00033,0.00167,0.19,0.0267,...,9,0,0,0,3,5,5,7,7,0
4202,3003,3602,19.22,16.25,0.30,100.09,0.000716,0.00167,0.11,0.0334,...,4,0,1,0,6,6,9,3,6,0
4203,3603,4202,19.89,13.82,0,99.40,0.000233,0.00167,0.12,0.0267,...,7,0,0,0,4,6,9,6,6,0
4225,16808,17408,35.21,34.58,4.22,83.01,0.000912,0.00167,1.00,0.015,...,4,4,5,2,7,9,9,4,3,2


Best solution for this appears to be a weighted t-test

### How to choose the weights?
Each individual being given a weight of 1 is wrong, since they have multiple datapoint.
Instead I tried to make sure that each individual is downweighted, so the weight sum between individuals is identical.



. Hence, the weight of each datapoint is inverse to the number individual's datapoint.

In [ ]:
def apply_data_weight(data, weight_by_col='id'):
    "Adds a test_wight column to dataframe. Also shows weightings applied to each group"
    

    data["test_weight"] = data.groupby(weight_by_col)[weight_by_col].transform("count")
    data["test_weight"] =  data["test_weight"].min() /data["test_weight"]

    weight_comparison = pd.DataFrame({
        'n_data': data.groupby(weight_by_col).size(),
        'post_weight_df': data.groupby(weight_by_col)["test_weight"].sum().astype(int),
        'datum_weight': data.groupby(weight_by_col)["test_weight"].mean()
    })
    print("The data was weighted as follows:")
    display(weight_comparison)
    return data

data= apply_data_weight(data, 'id')
print(len(data.id.unique()))

The data was weighted as follows:


,n_data,post_weight_df,datum_weight
id,,,
19070921,55,55,1
19072205,76,55,0.72
19072214,130,55,0.42
19072938,100,55,0.55
19072939,92,55,0.60
19072940,126,55,0.44
19080106,117,55,0.47
19081506,145,55,0.38
19082406,61,55,0.90


26


In [ ]:
import statsmodels.stats.weightstats as smw
import numpy as np
from statsmodels.stats.multitest import multipletests


pd.options.display.float_format = lambda x: f'{x:.2f}' if 0.1 <= abs(x) <= 100 else f'{x:.2e}'

def weighted_t_test(df, column, mask, weight_column='test_weight'):
    values_1 = df[mask][column]
    values_2 = df[~mask][column]

    weights_1 = df[mask][weight_column]
    weights_2 = df[~mask][weight_column]



    t_stat, p_value, df = smw.ttest_ind(
        values_1,
        values_2,
        usevar="unequal",
        weights=(weights_1, weights_2))
    
    weighted_mean_1 = np.average(values_1, weights=weights_1)
    weighted_mean_2 = np.average(values_2, weights=weights_2)
    
    weighted_std_1 = np.sqrt(np.average((values_1 - weighted_mean_1)**2, weights=weights_1))
    weighted_std_2 = np.sqrt(np.average((values_2 - weighted_mean_2)**2, weights=weights_2))
    
    result = {
        'weight_mean_1': weighted_mean_1,
        'weight_mean_2': weighted_mean_2,
        'weighted_std_1': weighted_std_1,
        'weighted_std_2': weighted_std_2,
        't_stat': t_stat,
        'df': df,
        'p_value': p_value,
    }
    
    return result


def metric_weighted_test(data, metrics, mask, weight_column='test_weight'):
    results = []
    for metric in metrics:
        result = weighted_t_test(data, metric, mask=mask)
        result['metric'] = metric
        results.append(result)
    result_df = pd.DataFrame(results)
    result_df.set_index('metric', inplace=True)
    return result_df



PDN_mask = data["DPN"] == True
result_df = metric_weighted_test(data, indicator_columns, mask=PDN_mask)

display(result_df)


,weight_mean_1,weight_mean_2,weighted_std_1,weighted_std_2,t_stat,df,p_value
metric,,,,,,,
SDRR,46.02,82.64,38.74,52.07,-12.14,3.78e+02,6.92e-29
RMSSD,39.30,78.88,45.98,66.14,-10.87,4.04e+02,2.67e-24
pNN50 (%),4.11,13.08,7.43,17.00,-12.80,7.16e+02,6.27e-34
Mean HR (bpm),82.57,75.24,9.76,11.22,9.98,3.33e+02,1.09e-20
VLF Power,1.70,5.44,3.45,7.63,-11.69,6.82e+02,6.79e-29
VLF Peak Frequency,2.83e-02,3.04e-02,8.43e-03,6.76e-03,-3.42,2.72e+02,7.30e-04
LF Power,34.20,1.17e+02,79.93,1.94e+02,-10.69,7.77e+02,5.52e-25
LF Peak Frequency,9.59e-02,0.11,3.70e-02,3.21e-02,-5.23,2.82e+02,3.31e-07
HF Power,5.10e+02,1.76e+03,1.26e+03,3.13e+03,-10.13,7.99e+02,9.11e-23



## multiple test correction
Since multiple values were processed, it is necessary to account for a false positive discovery rate.

Bonferroni correction:
* Is very conservative.
* Dead easy to code
    * Does not matter since I'm using statsmodels library

Benjamini-Yekutieli (fdr_by):
* More conservative (fewer discoveries) but ensures stronger error control in correlated data.
* remains valid under arbitrary dependence (e.g., correlated test statistics).
* Compared to methods like Bonferroni correction, which control FWER, BY correction is less conservative and preserves more statistical power.
    * This allows researchers to detect more true positives while still controlling for false discoveries.
* The main drawback of the BY correction is that it is more conservative than BH, meaning it reduces the number of discoveries slightly.

In [ ]:
# Apply Benjamini-Hochberg correction


def p_hacking_correction(result_df, alpha=0.05, method = 'bonferroni'):

    result_df[f'p_{method}'] = multipletests(result_df['p_value'], method= method)[1]
    result_df['p_adj<0.05'] = result_df[f'p_{method}'] < alpha
    return result_df

result_df = p_hacking_correction(result_df, 0.05, 'fdr_by')

We than have a separate function to make a more concise test result.

In [ ]:
def display_test_result_df(result_df):
    result_df = result_df.copy()

    rename = {
        'weight_mean_1': "PDN_mean",
        'weight_mean_2': "Diabetes_mean",
        'weighted_std_1': "PDN_std",
        'weighted_std_2': "Diabetes_std",
    }

    result_df['df'] = result_df['df'].astype(int)
    result_df['t_stat'] = result_df['t_stat'].astype(float).round(2)

    result_df.columns = pd.MultiIndex.from_tuples(
        [('Weighted Parameters', result_df.columns[0]),
         ('Weighted Parameters', result_df.columns[1]),
         ('Weighted Parameters', result_df.columns[2]),
         ('Weighted Parameters', result_df.columns[3]),
         ('Raw Statistics', result_df.columns[4]),
         ('Raw Statistics', result_df.columns[5]),
         ('Raw Statistics', result_df.columns[6]),
         ('Adjusted Statistics', result_df.columns[7]),
         ('Adjusted Statistics', result_df.columns[8])]
    )
    
    def float_format(x):
        if 0.01 <= abs(x) <= 1000:
            return f'{x:.2f}'
        elif 0.001 <= abs(x) < 0.01:
            return f'{x:.3f}'
        else:
            return f'{x:.2e}'
    
    pd.options.display.float_format = float_format

    result_df.rename(columns=rename, inplace=True)

    display(result_df)
    return result_df

#display_test_result_df(result_df).to_csv('processed_data/Benjamini-Yekutieli_adjusted_statistics_11-02-2025.csv')